In [1]:
import os
import fnmatch
import pandas as pd
import csv
import numpy as np
from pathlib import Path
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats
import matplotlib.pyplot as plt
import pingouin as pg

#pd.set_option('display.max_columns', 50)

In [2]:
# Definitions

analysis_path = os.path.dirname(os.getcwd())
path = os.path.dirname(analysis_path)

rat_list = ['Rat^{}'.format(i) for i in range(66, 90)]
rat_list_brain = ['Rat{}'.format(x) for x in pd.Index(np.arange(1,25)).astype(str).str.zfill(2)]

In [3]:
# load raw data sheet
df = pd.read_excel(f'{analysis_path}/00_blood_glucose/Rat_basic_data_with_calculations.xlsx', header=0, nrows=24)

In [4]:
df

,Rat_number,Group,Group_number,Glucose_level_1 (mmol/L),Glucose_level_2 (mmol/L),Glucose_level_3 (mmol/L),Weight (kg),Injected_dose (MBq),Glucose_level_2_scaled (mmol/L),Injected_dose_scaled (MBq),glucose_normalization_factor
0,Rat^66,Har + DMT,4,6.9,6.5,5.7,0.230,10.35,1.083333,9.553846,6
1,Rat^67,Har + DMT,4,7.4,6.3,8.0,0.204,9.19,1.050000,8.752381,6
2,Rat^68,Har,2,7.0,6.9,6.7,0.218,8.22,1.150000,7.147826,6
3,Rat^69,Har,2,5.9,6.1,6.2,0.221,11.69,1.016667,11.498361,6
4,Rat^70,Veh,1,6.9,6.2,6.4,0.212,8.40,1.033333,8.129032,6
5,Rat^71,Veh,1,5.6,6.0,7.2,0.230,10.23,1.000000,10.230000,6
6,Rat^72,DMT,3,6.9,6.5,6.8,0.218,11.25,1.083333,10.384615,6
7,Rat^73,DMT,3,5.6,5.8,6.4,0.204,10.37,0.966667,10.727586,6
8,Rat^74,DMT,3,5.7,5.8,6.2,0.224,8.12,0.966667,8.400000,6
9,Rat^75,DMT,3,6.5,4.8,7.2,0.227,10.00,0.800000,12.500000,6


In [5]:
ls_group = ['Veh', 'Har',  'DMT', 'Har + DMT']

In [6]:
##statistical calculations
# mean and sem of all activities per group
stats_of_components = []
mean_har_dmt_1 = []
mean_dmt_1 = []
mean_har_1 = []
mean_veh_1 = []

mean_har_dmt_2 = []
mean_dmt_2 = []
mean_har_2 = []
mean_veh_2 = []

mean_har_dmt_3 = []
mean_dmt_3 = []
mean_har_3 = []
mean_veh_3 = []

#for group in ls_group:
for group in ls_group:
    mean_gluc_1_group = np.round(df.loc[(df['Group']==group)][df.columns[3]].mean(), decimals = 2)
    std_gluc_1_group = np.round(df.loc[(df['Group']==group)][df.columns[3]].std(), decimals = 2)
    sem_gluc_1_group = np.round(df.loc[(df['Group']==group)][df.columns[3]].sem(), decimals = 2)
    
    mean_gluc_2_group = np.round(df.loc[(df['Group']==group)][df.columns[4]].mean(), decimals = 2)
    std_gluc_2_group = np.round(df.loc[(df['Group']==group)][df.columns[4]].std(), decimals = 2)
    sem_gluc_2_group = np.round(df.loc[(df['Group']==group)][df.columns[4]].sem(), decimals = 2)
    
    mean_gluc_3_group = np.round(df.loc[(df['Group']==group)][df.columns[5]].mean(), decimals = 2)
    std_gluc_3_group = np.round(df.loc[(df['Group']==group)][df.columns[5]].std(), decimals = 2)
    sem_gluc_3_group = np.round(df.loc[(df['Group']==group)][df.columns[5]].sem(), decimals = 2)
    
    
    stats_of_components_row = pd.DataFrame(data=[group, mean_gluc_1_group, std_gluc_1_group, sem_gluc_1_group,
                                                mean_gluc_2_group, std_gluc_2_group, sem_gluc_2_group,
                                                mean_gluc_3_group, std_gluc_3_group, sem_gluc_3_group]).T
    stats_of_components_row.columns = ['Treatment_Group', 'Concentration_1', 'STD_1', 'SEM_1',
                                      'Concentration_2', 'STD_2', 'SEM_2',
                                      'Concentration_3', 'STD_3', 'SEM_3']

    stats_of_components.append(stats_of_components_row)

    
### timepoint 1
mean = np.round(df.loc[(df['Group']=='Veh')][df.columns[3]].mean(), decimals = 1)
sd = np.round(df.loc[(df['Group']=='Veh')][df.columns[3]].std(), decimals = 1)
veh_1 = f'{mean} ({sd})'
mean_veh_1.append(veh_1)

mean = np.round(df.loc[(df['Group']=='DMT')][df.columns[3]].mean(), decimals = 1)
sd = np.round(df.loc[(df['Group']=='DMT')][df.columns[3]].std(), decimals = 1)
dmt_1 = f'{mean} ({sd})'
mean_dmt_1.append(dmt_1)

mean = np.round(df.loc[(df['Group']=='Har')][df.columns[3]].mean(), decimals = 1)
sd = np.round(df.loc[(df['Group']=='Har')][df.columns[3]].std(), decimals = 1)
har_1 = f'{mean} ({sd})'
mean_har_1.append(har_1)

mean = np.round(df.loc[(df['Group']=='Har + DMT')][df.columns[3]].mean(), decimals = 1)
sd = np.round(df.loc[(df['Group']=='Har + DMT')][df.columns[3]].std(), decimals = 1)
har_dmt_1 = f'{mean} ({sd})'
mean_har_dmt_1.append(har_dmt_1)

## timepoint 2
mean = np.round(df.loc[(df['Group']=='Veh')][df.columns[4]].mean(), decimals = 1)
sd = np.round(df.loc[(df['Group']=='Veh')][df.columns[4]].std(), decimals = 1)
veh_2 = f'{mean} ({sd})'
mean_veh_2.append(veh_2)

mean = np.round(df.loc[(df['Group']=='DMT')][df.columns[4]].mean(), decimals = 1)
sd = np.round(df.loc[(df['Group']=='DMT')][df.columns[4]].std(), decimals = 1)
dmt_2 = f'{mean} ({sd})'
mean_dmt_2.append(dmt_2)

mean = np.round(df.loc[(df['Group']=='Har')][df.columns[4]].mean(), decimals = 1)
sd = np.round(df.loc[(df['Group']=='Har')][df.columns[4]].std(), decimals = 1)
har_2 = f'{mean} ({sd})'
mean_har_2.append(har_2)

mean = np.round(df.loc[(df['Group']=='Har + DMT')][df.columns[4]].mean(), decimals = 1)
sd = np.round(df.loc[(df['Group']=='Har + DMT')][df.columns[4]].std(), decimals = 1)
har_dmt_2 = f'{mean} ({sd})'
mean_har_dmt_2.append(har_dmt_2)

## timepoint 3
mean = np.round(df.loc[(df['Group']=='Veh')][df.columns[5]].mean(), decimals = 1)
sd = np.round(df.loc[(df['Group']=='Veh')][df.columns[5]].std(), decimals = 1)
veh_3 = f'{mean} ({sd})'
mean_veh_3.append(veh_3)

mean = np.round(df.loc[(df['Group']=='DMT')][df.columns[5]].mean(), decimals = 1)
sd = np.round(df.loc[(df['Group']=='DMT')][df.columns[5]].std(), decimals = 1)
dmt_3 = f'{mean} ({sd})'
mean_dmt_3.append(dmt_3)

mean = np.round(df.loc[(df['Group']=='Har')][df.columns[5]].mean(), decimals = 1)
sd = np.round(df.loc[(df['Group']=='Har')][df.columns[5]].std(), decimals = 1)
har_3 = f'{mean} ({sd})'
mean_har_3.append(har_3)

mean = np.round(df.loc[(df['Group']=='Har + DMT')][df.columns[5]].mean(), decimals = 1)
sd = np.round(df.loc[(df['Group']=='Har + DMT')][df.columns[5]].std(), decimals = 1)
har_dmt_3 = f'{mean} ({sd})'
mean_har_dmt_3.append(har_dmt_3)

stats_of_components = pd.concat(stats_of_components, axis=0)
stats_of_components.to_excel(f'{analysis_path}/00_blood_glucose/00_Results_glucose_concentration.xlsx', sheet_name="stats", index = False)


In [7]:
##statistical calculations
# mean and sem of all activities per group
stats_of_components = []
mean_timepoint_1 = []
mean_timepoint_2 = []
mean_timepoint_3 = []
group_list = []

#for group in ls_group:
for group in ls_group:
    mean_gluc_1_group = np.round(df.loc[(df['Group']==group)][df.columns[3]].mean(), decimals = 2)
    std_gluc_1_group = np.round(df.loc[(df['Group']==group)][df.columns[3]].std(), decimals = 2)
    sem_gluc_1_group = np.round(df.loc[(df['Group']==group)][df.columns[3]].sem(), decimals = 2)
    
    mean_gluc_2_group = np.round(df.loc[(df['Group']==group)][df.columns[4]].mean(), decimals = 2)
    std_gluc_2_group = np.round(df.loc[(df['Group']==group)][df.columns[4]].std(), decimals = 2)
    sem_gluc_2_group = np.round(df.loc[(df['Group']==group)][df.columns[4]].sem(), decimals = 2)
    
    mean_gluc_3_group = np.round(df.loc[(df['Group']==group)][df.columns[5]].mean(), decimals = 2)
    std_gluc_3_group = np.round(df.loc[(df['Group']==group)][df.columns[5]].std(), decimals = 2)
    sem_gluc_3_group = np.round(df.loc[(df['Group']==group)][df.columns[5]].sem(), decimals = 2)
    
    
    stats_of_components_row = pd.DataFrame(data=[group, mean_gluc_1_group, std_gluc_1_group, sem_gluc_1_group,
                                                mean_gluc_2_group, std_gluc_2_group, sem_gluc_2_group,
                                                mean_gluc_3_group, std_gluc_3_group, sem_gluc_3_group]).T
    stats_of_components_row.columns = ['Treatment_Group', 'Concentration_1', 'STD_1', 'SEM_1',
                                      'Concentration_2', 'STD_2', 'SEM_2',
                                      'Concentration_3', 'STD_3', 'SEM_3']

    stats_of_components.append(stats_of_components_row)

    
    ### timepoints 1,2,3 order: Veh, Har, DMT, Har + DMT
    mean = np.round(df.loc[(df['Group']==group)][df.columns[3]].mean(), decimals = 2)
    sd = np.round(df.loc[(df['Group']==group)][df.columns[3]].std(), decimals = 2)
    timepoint_1 = f'{mean} ({sd})'
    mean_timepoint_1.append(timepoint_1)

    mean = np.round(df.loc[(df['Group']==group)][df.columns[4]].mean(), decimals = 2)
    sd = np.round(df.loc[(df['Group']==group)][df.columns[4]].std(), decimals = 2)
    timepoint_2 = f'{mean} ({sd})'
    mean_timepoint_2.append(timepoint_2)

    mean = np.round(df.loc[(df['Group']==group)][df.columns[5]].mean(), decimals = 2)
    sd = np.round(df.loc[(df['Group']==group)][df.columns[5]].std(), decimals = 2)
    timepoint_3 = f'{mean} ({sd})'
    mean_timepoint_3.append(timepoint_3)
    
    group_list.append(group)
    


stats_of_components = pd.concat(stats_of_components, axis=0)
stats_of_components.to_excel(f'{analysis_path}/00_blood_glucose/00_Results_glucose_concentration.xlsx', sheet_name="stats", index = False)


## Check if glucose values are not differing between timepoints groupwise


### Normality testing with shapiro-wilk test


In [8]:
for group in ls_group:
    df_group = df[df['Group']==group].iloc[:, 3:6]

    print(group)
    print('Timepoint 1:    ', stats.shapiro(df_group.iloc[:,0]))
    print('Timepoint 2:    ', stats.shapiro(df_group.iloc[:,1]))
    print('Timepoint 3:    ', stats.shapiro(df_group.iloc[:,2]))
    print('_______')
    print('')
    #(region_dict[f'{region}_lm'].resid)

Veh
Timepoint 1:     ShapiroResult(statistic=0.9722484946250916, pvalue=0.907184362411499)
Timepoint 2:     ShapiroResult(statistic=0.8783153295516968, pvalue=0.261409729719162)
Timepoint 3:     ShapiroResult(statistic=0.9428593516349792, pvalue=0.6823129653930664)
_______

Har
Timepoint 1:     ShapiroResult(statistic=0.8770363926887512, pvalue=0.2557218372821808)
Timepoint 2:     ShapiroResult(statistic=0.9681164026260376, pvalue=0.8795509934425354)
Timepoint 3:     ShapiroResult(statistic=0.9489880204200745, pvalue=0.7320792078971863)
_______

DMT
Timepoint 1:     ShapiroResult(statistic=0.8786189556121826, pvalue=0.26277482509613037)
Timepoint 2:     ShapiroResult(statistic=0.9036424160003662, pvalue=0.39590778946876526)
Timepoint 3:     ShapiroResult(statistic=0.9292192459106445, pvalue=0.5740861296653748)
_______

Har + DMT
Timepoint 1:     ShapiroResult(statistic=0.9504017233848572, pvalue=0.7435342669487)
Timepoint 2:     ShapiroResult(statistic=0.8910704851150513, pvalue=0.3238

### Testing Homogeneity of Variance

In [9]:
# homogeneity of variance testing

for group in ls_group:
    df_group = df[df['Group']==group].iloc[:, 3:6]
    
    print(group)
    print(stats.levene(df_group.iloc[:,0],
                                           df_group.iloc[:,1],
                                           df_group.iloc[:,2]))
    print('_______')
    print('')


Veh
LeveneResult(statistic=1.6194690265486724, pvalue=0.23077450638125124)
_______

Har
LeveneResult(statistic=0.7460732984293215, pvalue=0.49102719290073504)
_______

DMT
LeveneResult(statistic=0.020387359836901216, pvalue=0.9798461589328911)
_______

Har + DMT
LeveneResult(statistic=0.11538461538461556, pvalue=0.891806551015499)
_______



-> no assumption violations found -> continue with ANOVA

### Perform ANOVA for glucose per group

In [10]:
f_stats = []
p_vals = []
degfs = []
eta_squared =[]

for group in ls_group:
    df_group = df[df['Group']==group].iloc[:, 3:6]

    # test for homogeneity of variance:
    print(group)
    print('-----')
    print(stats.levene(df_group.iloc[:,0],
             df_group.iloc[:,1],
             df_group.iloc[:,2]))
    
    print(pg.rm_anova(df_group, detailed=True))
    print('-----')
    print('')
    f_stat = str(np.round(pg.rm_anova(df_group, detailed=True).F[0], decimals=2))
    f_stats.append(f_stat)
    p_val = str(np.round(pg.rm_anova(df_group, detailed=True).iloc[0, 5], decimals=2))
    p_vals.append(p_val)
    degf = f'({pg.rm_anova(df_group, detailed=True).DF[0]}, {pg.rm_anova(df_group, detailed=True).DF[1]})'
    degfs.append(degf)
    eta_sq = str(np.round(pg.rm_anova(df_group, detailed=True).iloc[0, 6], decimals=2))
    eta_squared.append(eta_sq)
    #pg.rm_anova(df_group, detailed=True)

Veh
-----
LeveneResult(statistic=1.6194690265486724, pvalue=0.23077450638125124)
   Source        SS  DF        MS        F    p-unc       ng2       eps
0  Within  0.174444   2  0.087222  0.16649  0.84893  0.020756  0.595842
1   Error  5.238889  10  0.523889      NaN      NaN       NaN       NaN
-----

Har
-----
LeveneResult(statistic=0.7460732984293215, pvalue=0.49102719290073504)
   Source        SS  DF        MS         F     p-unc      ng2       eps
0  Within  3.134444   2  1.567222  2.939154  0.099078  0.25201  0.670813
1   Error  5.332222  10  0.533222       NaN       NaN      NaN       NaN
-----



/Users/Edge/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pingouin/parametric.py:551: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  ss_resall = grp_with.apply(lambda x: (x - x.mean()) ** 2).sum()
/Users/Edge/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pingouin/parametric.py:551: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

DMT
-----
LeveneResult(statistic=0.020387359836901216, pvalue=0.9798461589328911)
   Source        SS  DF        MS         F     p-unc       ng2       eps
0  Within  0.671111   2  0.335556  0.974822  0.410417  0.111645  0.661002
1   Error  3.442222  10  0.344222       NaN       NaN       NaN       NaN
-----

Har + DMT
-----
LeveneResult(statistic=0.11538461538461556, pvalue=0.891806551015499)
   Source        SS  DF        MS         F     p-unc       ng2       eps
0  Within  1.401111   2  0.700556  2.089824  0.174451  0.112892  0.965339
1   Error  3.352222  10  0.335222       NaN       NaN       NaN       NaN
-----



/Users/Edge/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pingouin/parametric.py:551: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  ss_resall = grp_with.apply(lambda x: (x - x.mean()) ** 2).sum()
/Users/Edge/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pingouin/parametric.py:551: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

In [11]:
### for latex table

df_glucose = pd.DataFrame([group_list]).T
df_glucose = df_glucose.set_axis([''], axis='columns', copy=False)

df_glucose.insert(loc=1, column='\textbf{T1}', value=list(map(lambda x: '{' + x +'}',mean_timepoint_1)))
df_glucose.insert(loc=2, column='\textbf{T2}', value=list(map(lambda x: '{' + x +'}', mean_timepoint_2)))
df_glucose.insert(loc=3, column='\textbf{T3}', value=list(map(lambda x: '{' + x +'}', mean_timepoint_3)))
df_glucose.insert(loc=4, column='\textbf{\textit{F}}', value=list(map(lambda x: '{' + x +'}', f_stats)))
df_glucose.insert(loc=5, column='\textbf{\textit{p}}', value=list(map(lambda x: '{' + x +'}', p_vals)))
df_glucose.insert(loc=6, column='\textbf{df}', value=list(map(lambda x: '{' + x +'}', degfs)))
df_glucose.insert(loc=7, column='\textbf{\textit{$\eta^{2}$}}', value=list(map(lambda x: '{' + x +'}', eta_squared)))

eta_squared

df_glucose_xlsx = pd.DataFrame([group_list]).T
df_glucose_xlsx.set_axis([''], axis='columns', copy=False)

df_glucose_xlsx.insert(loc=1, column='T1', value=mean_timepoint_1)
df_glucose_xlsx.insert(loc=2, column='T2', value=mean_timepoint_2)
df_glucose_xlsx.insert(loc=3, column='T3', value=mean_timepoint_3)
df_glucose_xlsx.insert(loc=4, column='F', value=f_stats)
df_glucose_xlsx.insert(loc=5, column='p', value=p_vals)
df_glucose_xlsx.insert(loc=6, column='df', value=degfs)
df_glucose_xlsx.insert(loc=7, column='eta^2', value=eta_squared)

df_glucose_xlsx.to_excel(f'{analysis_path}/00_blood_glucose/stats_table_blood_glucose.xlsx', sheet_name="blood_glucose", index = False)
df_glucose_xlsx.to_excel(f'{analysis_path}/03_statistics/stats_table_blood_glucose.xlsx', sheet_name="blood_glucose", index = False)


df_glucose


,,\textbf{T1},\textbf{T2},\textbf{T3},\textbf{\textit{F}},\textbf{\textit{p}},\textbf{df},\textbf{\textit{$\eta^{2}$}}
0,Veh,{5.92 (0.74)},{5.72 (0.48)},{5.93 (0.93)},{0.17},{0.85},"{(2, 10)}",{0.02}
1,Har,{6.55 (0.73)},{6.97 (0.7)},{5.95 (0.91)},{2.94},{0.1},"{(2, 10)}",{0.25}
2,DMT,{6.17 (0.54)},{5.87 (0.65)},{6.33 (0.6)},{0.97},{0.41},"{(2, 10)}",{0.11}
3,Har + DMT,{6.37 (0.88)},{5.78 (0.75)},{6.38 (0.93)},{2.09},{0.17},"{(2, 10)}",{0.11}


In [12]:
# Write to file
with open(f'{analysis_path}/03_statistics/stats_table_blood_glucose.tbl', "w") as tbl:

    format = "l" + \
        "@{\hskip 12pt}" +\
        8*"S[table-format = 2.2]"



    tbl.write(df_glucose.to_latex(index=False,
                      escape=False,
                      column_format=format)
            )

/var/folders/lb/nxxqtlbj7yd1c5frnp5_vxk40000gn/T/ipykernel_93816/1625938631.py:10: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  tbl.write(df_glucose.to_latex(index=False,
